In [79]:
# Imports
import pandas as pd
import numpy as np

from pathlib import Path

import hvplot.pandas
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report


from random import randrange 
from datetime import timedelta 
from datetime import datetime 

import csv
from pathlib import Path

import time
import numpy
import io

import os
import requests
import json
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from dotenv import load_dotenv

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [80]:
# Import dataset into a Pandas Dataframe
data_df = pd.read_csv(
    Path("data_set_1.csv") 
#    index_col='Date', 
#    infer_datetime_format=True, 
#    parse_dates=True
)

# Review the DataFrame
data_df.head()

,Name,Date,Time In,Time Out,Full Time/Part Time,Hours Allocated
0,Person 1,NaN,05:35,08:57,Part Time,160
1,Person 2,NaN,08:14,05:29,Full Time,217
2,Person 3,NaN,11:14,06:24,Full Time,224
3,Person 4,NaN,10:51,08:13,Part Time,84
4,Person 5,NaN,09:06,02:49,Part Time,148


In [81]:
#Data Preparation
data_df = data_df.drop(columns=['Date'])
data_df = data_df.drop(columns=['Time In'])
data_df = data_df.drop(columns=['Time Out'])
display(data_df)

,Name,Full Time/Part Time,Hours Allocated
0,Person 1,Part Time,160
1,Person 2,Full Time,217
2,Person 3,Full Time,224
3,Person 4,Part Time,84
4,Person 5,Part Time,148
...,...,...,...
994,Person 995,Full Time,231
995,Person 996,Full Time,190
996,Person 997,Part Time,82
997,Person 998,Full Time,163


In [82]:
#Creating additional pseduodata

rng = np.random.default_rng()

#Temporary hardcode
df = pd.DataFrame(rng.integers(1, 12, size=(999, 1)), columns=list('A'))

#Review
display(df)

,A
0,9
1,10
2,10
3,7
4,9
...,...
994,8
995,10
996,6
997,8


In [83]:
#Rename Column 
df_2 = df.rename({'A':'Hours Worked'}, axis = 1)

df_2

,Hours Worked
0,9
1,10
2,10
3,7
4,9
...,...
994,8
995,10
996,6
997,8


In [84]:
#Concatentate dataframes

new_df = pd.concat([data_df,df_2],axis = 1)

#review data

new_df

,Name,Full Time/Part Time,Hours Allocated,Hours Worked
0,Person 1,Part Time,160,9
1,Person 2,Full Time,217,10
2,Person 3,Full Time,224,10
3,Person 4,Part Time,84,7
4,Person 5,Part Time,148,9
...,...,...,...,...
994,Person 995,Full Time,231,8
995,Person 996,Full Time,190,10
996,Person 997,Part Time,82,6
997,Person 998,Full Time,163,8


In [85]:
#Set dataframe index to DateTime 

new_df.index = pd.to_datetime(data_df.index)
new_df

,Name,Full Time/Part Time,Hours Allocated,Hours Worked
1970-01-01 00:00:00.000000000,Person 1,Part Time,160,9
1970-01-01 00:00:00.000000001,Person 2,Full Time,217,10
1970-01-01 00:00:00.000000002,Person 3,Full Time,224,10
1970-01-01 00:00:00.000000003,Person 4,Part Time,84,7
1970-01-01 00:00:00.000000004,Person 5,Part Time,148,9
...,...,...,...,...
1970-01-01 00:00:00.000000994,Person 995,Full Time,231,8
1970-01-01 00:00:00.000000995,Person 996,Full Time,190,10
1970-01-01 00:00:00.000000996,Person 997,Part Time,82,6
1970-01-01 00:00:00.000000997,Person 998,Full Time,163,8


In [86]:
#Define Random Function

def random_dates(start, end, n, unit='D', seed=None):
    if not seed:
        np.random.seed(0)

    ndays = (end - start).days + 1
    return pd.to_timedelta(np.random.rand(n) * ndays, unit=unit) + start
#Output will produce a datetime64 object 

np.random.seed(0)

#2022 Fiscal Q4
start = pd.to_datetime('2022-10-01')
end = pd.to_datetime('2022-12-31')

#Instantiate 
random_date = random_dates(start, end, 999)

#Review
random_date

DatetimeIndex(['2022-11-20 11:46:48.779990400',
               '2022-12-05 19:08:17.235398399',
               '2022-11-25 10:54:05.523753600',
               '2022-11-20 03:06:07.445030400',
               '2022-11-08 23:25:47.268969600',
               '2022-11-29 10:08:03.125932800',
               '2022-11-10 06:11:33.224870400',
               '2022-12-22 01:02:05.228620800',
               '2022-12-28 15:46:02.550662400',
               '2022-11-05 06:38:19.944844800',
               ...
               '2022-12-10 22:51:56.960035200',
               '2022-10-02 02:52:26.092329600',
               '2022-10-30 16:48:27.428025600',
               '2022-10-22 02:53:05.704396800',
               '2022-11-16 15:09:12.284438400',
                  '2022-12-07 18:58:18.411600',
                  '2022-10-09 23:40:09.909552',
               '2022-11-17 08:56:53.598739200',
               '2022-12-26 08:00:49.478054400',
                  '2022-10-22 00:51:05.704416'],
              dtype=

In [87]:
#Set index as punches

new_df.index = random_date

#Review 
new_df

,Name,Full Time/Part Time,Hours Allocated,Hours Worked
2022-11-20 11:46:48.779990400,Person 1,Part Time,160,9
2022-12-05 19:08:17.235398399,Person 2,Full Time,217,10
2022-11-25 10:54:05.523753600,Person 3,Full Time,224,10
2022-11-20 03:06:07.445030400,Person 4,Part Time,84,7
2022-11-08 23:25:47.268969600,Person 5,Part Time,148,9
...,...,...,...,...
2022-12-07 18:58:18.411600000,Person 995,Full Time,231,8
2022-10-09 23:40:09.909552000,Person 996,Full Time,190,10
2022-11-17 08:56:53.598739200,Person 997,Part Time,82,6
2022-12-26 08:00:49.478054400,Person 998,Full Time,163,8


In [ ]:
## PART 2: Clean Data

In [88]:
#Review Dataframe
new_df

,Name,Full Time/Part Time,Hours Allocated,Hours Worked
2022-11-20 11:46:48.779990400,Person 1,Part Time,160,9
2022-12-05 19:08:17.235398399,Person 2,Full Time,217,10
2022-11-25 10:54:05.523753600,Person 3,Full Time,224,10
2022-11-20 03:06:07.445030400,Person 4,Part Time,84,7
2022-11-08 23:25:47.268969600,Person 5,Part Time,148,9
...,...,...,...,...
2022-12-07 18:58:18.411600000,Person 995,Full Time,231,8
2022-10-09 23:40:09.909552000,Person 996,Full Time,190,10
2022-11-17 08:56:53.598739200,Person 997,Part Time,82,6
2022-12-26 08:00:49.478054400,Person 998,Full Time,163,8


In [89]:
#Review categories
display(new_df.columns)
display(new_df.info())

Index(['Name', 'Full Time/Part Time', 'Hours Allocated', 'Hours Worked'], dtype='object')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 999 entries, 2022-11-20 11:46:48.779990400 to 2022-10-22 00:51:05.704416
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Name                 999 non-null    object
 1   Full Time/Part Time  999 non-null    object
 2   Hours Allocated      999 non-null    int64 
 3   Hours Worked         999 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 39.0+ KB


None

In [90]:
#Divide data between Part-Time and Full-Time

new_df['Full Time/Part Time'].value_counts()

Part Time    528
Full Time    471
Name: Full Time/Part Time, dtype: int64

In [91]:
#Prepare data further for machine learning code 
new_df = new_df.drop(columns=['Name'])

#Review
new_df

,Full Time/Part Time,Hours Allocated,Hours Worked
2022-11-20 11:46:48.779990400,Part Time,160,9
2022-12-05 19:08:17.235398399,Full Time,217,10
2022-11-25 10:54:05.523753600,Full Time,224,10
2022-11-20 03:06:07.445030400,Part Time,84,7
2022-11-08 23:25:47.268969600,Part Time,148,9
...,...,...,...
2022-12-07 18:58:18.411600000,Full Time,231,8
2022-10-09 23:40:09.909552000,Full Time,190,10
2022-11-17 08:56:53.598739200,Part Time,82,6
2022-12-26 08:00:49.478054400,Full Time,163,8


In [92]:
#Transform categorical variables

#Create Instance
enc = OneHotEncoder(handle_unknown='ignore')
#Fit instance to categorical data
enc.fit(new_df[['Full Time/Part Time']])

#Transform data and create DataFrame
categories = enc.transform(new_df[['Full Time/Part Time']])

#Create Pandas Dataframe

output = pd.DataFrame(categories.toarray(),columns = enc.categories_)
#Review dataframe
display(output)

,Full Time,Part Time
0,0.0,1.0
1,1.0,0.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0
...,...,...
994,1.0,0.0
995,1.0,0.0
996,0.0,1.0
997,1.0,0.0
